In [ ]:
# Imports
import webrtcvad
import librosa
import numpy as np
import sounddevice as sd
import wave
from IPython.display import Audio, display
import matplotlib.pyplot as plt


In [ ]:
def process_and_save_audio(
        file_path, 
        vad_aggressiveness=2, 
        save_path=None, 
        show_plot=False, 
        play_audio=False):
    """
    Обрабатывает аудиофайл для выделения сегментов с речью, воспроизводит, отображает и сохраняет результат.

    Параметры:
    - file_path (str): Путь к входному аудиофайлу.
    - vad_aggressiveness (int): Уровень агрессивности VAD (0-3).
    - save_path (str): Путь для сохранения выходного аудиофайла. Если None, файл не сохраняется.
    - show_plot (bool): Отображение графика аудио (True/False).
    - play_audio (bool): Воспроизведение аудио (True/False).
    """
    try:
        # Загрузка аудиофайла и проверка его длины
        y, sr = librosa.load(file_path, sr=16000)
        if len(y) == 0:
            raise ValueError("Аудиофайл пустой или не был загружен корректно.")
        
        # Настройка VAD и параметры окна
        vad = webrtcvad.Vad(vad_aggressiveness)
        window_duration = 0.01  # 10 мс окно
        samples_per_window = int(window_duration * sr)

        # Проход по аудиоданным и выделение сегментов с голосом
        segments = []
        for start in range(0, len(y), samples_per_window):
            end = start + samples_per_window
            segment = y[start:end]

            # Пропускаем сегменты, которые не соответствуют длине окна
            if len(segment) != samples_per_window:
                continue

            # Преобразование сегмента в int16 и проверка с помощью VAD
            segment = (segment * 32768).astype(np.int16)
            is_speech = vad.is_speech(segment.tobytes(), sample_rate=sr)

            if is_speech:
                segments.extend(segment)
        
        # Проверка, есть ли выделенные голосовые сегменты
        if len(segments) == 0:
            raise ValueError("Не удалось обнаружить речь в аудиофайле.")
        
        # Преобразование сегментов для воспроизведения и сохранения
        audio_data = np.array(segments) / 32768.0  # нормализация для float32

        # Воспроизведение аудио
        if play_audio:
            print("Воспроизведение сегментов с голосом...")
            sd.play(audio_data, samplerate=sr)
            sd.wait()

        # Отображение аудиосигнала
        if show_plot:
            plt.figure(figsize=(10, 4))
            plt.plot(np.linspace(0, len(audio_data) / sr, len(audio_data)), audio_data)
            plt.title("Сегменты с голосом")
            plt.xlabel("Время (с)")
            plt.ylabel("Амплитуда")
            plt.show()

        # Сохранение в файл
        if save_path:
            with wave.open(save_path, "w") as wf:
                wf.setnchannels(1)  # моно
                wf.setsampwidth(2)  # 16-битный int
                wf.setframerate(sr)
                wf.writeframes(np.array(segments, dtype=np.int16).tobytes())
            print(f"Файл сохранён по пути: {save_path}")
        
        # Воспроизведение в Jupyter
        if play_audio and 'IPython' in globals():
            display(Audio(data=audio_data, rate=sr))
        
        return audio_data

    except Exception as e:
        print("Ошибка:", e)
        raise e